# Imports

In [3]:
# From Imports
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
from pandas import DataFrame
from datetime import datetime
from scipy.optimize import minimize
from arch import arch_model
from sklearn.linear_model import LinearRegression

# Alias Imports
import numpy_financial as npf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pmdarima as pm

# Imports
import math
import arch
import openpyxl 
import pprint

print("Import Complete")

Import Complete


# Importing Uniswap Pool, LP, and USDC Lending Rate Data

In [ ]:
prices_100bp = []  
volumes_100bp = [] #total $USD volume of USDC and ETH traded on each block
avg_LP_spread_100bp = [] #probably need an intermediate step to calc this, avg_LP_upper - avg_LP_lower on each block interval
active_liq_depth_100bp = [] # sum(x) * sum (y), where the sums are based on the LP positions that contain the current price in their range


# Predicted Loss Function Calculations

In [ ]:
#Helper functions

#Used to calculate alpha_lu, alpha_d, and alpha_0
def alpha_t(k, Z, Zl, Zu)
        if Zl < Z < Zu
            xi = k * (Z**(1/2) - (Zl)**(1/2))
            yi = k * (Z**(-1/2) - Zu**(-1/2))
            alpha = xi + yi * Z
        elif Z < Zl:
            xi = 0
            yi = k * ((Zl)**(-1/2) - (Zu)**(-1/2))
            alpha = xi + yi * Z
        elif Z > Zu:
            xi = k * ((Zu)**(1/2) - (Zl)**(1/2))
            yi = 0
            alpha = xi + yi * Z
        return alpha

#See denominator or convexity cost
def LP_spread_conversion(Z, Zl, Zu):
    sigma_u = -2 * (Z**(1/2)) / (Zu**(1/2)) + 2
    sigma_l = -2 * (Z**(1/2)) / (Zl**(1/2)) + 2
    return sigma_u + sigma_l

#Quadratic Variation, needs multiple step ahead forecast, maybe generate 20 forecasts over the short block interval
def calc_QV(price_list):
    log_returns = np.diff(np.log(price_list))
    QV = []
    for i in range(10, len(log_returns), 10):
        QV.append(np.sum(np.diff(log_returns[i-10:i])**2))
    QV = pd.Series(QV)
    return QV
    

In [ ]:
#Combining calculations
#functionally very similar to LVR

Integral[alpha_lu / (2 * spread * Z**2) * QV[Z,Z]] + Integral[]



In [ ]:
# Calculate portfolio losses ONLY over here, fix 
portfolio_losses = []  # Calculate portfolio losses based on simulated_spreads and simulated_vwap
daily_profits = []  # Calculate daily profits based on simulated_spreads and simulated_vwap
daily_opportunity_cost = []  # Calculate daily opportunity cost
total_daily_pnl = []  # Calculate total daily PnL

for i in range(n_simulations):
    alpha_0 = alpha_t(k, bond_prices[0, 0], bond_prices[0, 0] - simulated_spreads[i, 0] / 2, bond_prices[0, 0] + simulated_spreads[i, 0] / 2)
    
    portfolio_loss_i = [alpha_0 - alpha_t(k, Z, Zl, Zu) for Z, Zl, Zu in zip(bond_prices[:, 1:], simulated_spreads[i, :] / 2, simulated_spreads[i, :] / 2)]
    daily_profits_i = forecasted_spreads * simulated_volumes[i, :] * fee_rate / 2  # Adjust for bid and ask
    
    # Calculate daily opportunity cost based on your formula
    daily_opportunity_cost_i = [integral_formula(alpha_0 - alpha_d, r, dt) for alpha_d, r, dt in zip(portfolio_loss_i, risk_free_rate_data, dt_values)]
    
    total_daily_pnl_i = np.array(portfolio_loss_i) + np.array(daily_opportunity_cost_i) + daily_profits_i
    
    portfolio_losses.append(portfolio_loss_i)
    daily_profits.append(daily_profits_i)
    daily_opportunity_cost.append(daily_opportunity_cost_i)
    total_daily_pnl.append(total_daily_pnl_i)

portfolio_losses = np.array(portfolio_losses)
daily_profits = np.array(daily_profits)
daily_opportunity_cost = np.array(daily_opportunity_cost)
total_daily_pnl = np.array(total_daily_pnl)

# Fitting and Forecasting Parameters using TS Models

In [ ]:
#Get TS parameters for optimal ARIMA GARCH model
prices_series = []

# Define the window size (half of the data)
window_size = len(prices_series) // 2

# Create lists to store optimal ARIMA and GARCH orders
optimal_arima_orders = []
optimal_garch_orders = []

# Iterate through the data with a sliding window
for i in range(0, len(prices_series) - window_size):
    # Extract the current window
    window = prices_series[i:i + window_size]

    # Perform ARIMA modeling to find the optimal order
    arima_model = pm.auto_arima(window, suppress_warnings=True)
    best_arima_order = arima_model.order
    optimal_arima_orders.append(best_arima_order)

    # Perform GARCH modeling to find the optimal order
    garch_model = arch.arch_model(window, vol='Garch', p=1, q=1)
    res = garch_model.fit(disp='off')
    best_garch_order = (res.model.p, res.model.q)
    optimal_garch_orders.append(best_garch_order)

# Now, you have lists of optimal ARIMA and GARCH orders for each window.

# Initialize lists to store forecasts and actual prices
forecasts = []
actual_prices = []

# Iterate through the second half of the data for one-step ahead forecasts
for i in range(window_size, len(prices_series)):
    # Extract the current window for ARIMA-GARCH modeling
    current_window = prices_series[i - window_size:i]

    # Use the optimal ARIMA and GARCH orders from the lists
    best_arima_order = optimal_arima_orders[i - window_size]
    best_garch_order = optimal_garch_orders[i - window_size]

    # Train the ARIMA-GARCH model using the current window
    # Perform one-step ahead forecast
    arima_model = pm.ARIMA(order=best_arima_order)
    arima_model.fit(current_window)
    garch_model = arch.arch_model(current_window, vol='Garch', p=best_garch_order[0], q=best_garch_order[1])
    res = garch_model.fit(disp='off')
    
    # Forecast the next price using ARIMA-GARCH
    forecast = arima_model.predict(n_periods=1, return_conf_int=False)
    volatility = res.conditional_volatility[-1]
    forecast = forecast * np.sqrt(volatility)  # Adjust the forecast with GARCH volatility
    forecasts.append(forecast[0])

    # Record the actual price for comparison
    actual_prices.append(prices_series[i])


